## DEPENDENCE INSTALL

- Dependencies install for evaluation of metrics
- Instalação das depedendencias necessárias para executar todos as avaliações

In [5]:
%pip install numpy cython
%pip install wheel
%pip install ir_datasets
%pip install scikit-learn
%pip install nltk
%pip install requests
%pip install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 12.0 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 17.9 MB/s eta 0:00:0000:0100:01
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python

In [1]:
import ir_datasets
import numpy as np
import requests
import json
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import dcg_score
from sklearn.metrics import ndcg_score


In [2]:
dataset = ir_datasets.load('msmarco-passage/train')

In [3]:
dataset.docs_count()

8841823

## EVALUETION FUNCTIONS

- 
- Implementação de funções em algoritimos para executar as avaliações


### Calculo do DCG@k

In [ ]:
def calc_dcg_k(relevance):
  result = 0
  for i in range(len(relevance)):
    log = np.log2(1 + (i + 1))
    result += (relevance[i] / log)
  return result

### Calculo NDCG@K

In [ ]:
def calc_ndcg_k(relevance, true_relevance):
  return calc_dcg_k(relevance) / calc_dcg_k(true_relevance)

#### Exemplos

In [ ]:
relevance = [3,3,0]
true_relevance = [4,3,3]

In [ ]:
calc_ndcg_k(relevance, true_relevance)

In [ ]:
ndcg_score(np.asarray([true_relevance]), np.asarray([relevance]))

## INDEXAR DOCUMENTOS

In [ ]:
url = 'https://bpmxju353h.execute-api.us-east-1.amazonaws.com/dev/neuralir'

In [ ]:
nltk.download("punkt")

In [ ]:
count = 0

for doc in dataset.docs_iter():

  sent = list(word_tokenize(doc.text))
  mystr = ' '.join(map(str,sent))

  myobj = {
            'id': doc.doc_id,
            'title': doc.title.strip(),
            'body': mystr.strip()
          }
  # x = requests.post(url, json = myobj)
  count += 1

print("Indexed ", count)

In [ ]:
dataset.docs_cls()

### Calculo do DCG@k

## EFETUAR BUSCAS

In [ ]:
documents = []

In [ ]:
url = "https://bpmxju353h.execute-api.us-east-1.amazonaws.com/dev/neuralir?query="

In [ ]:
for query in dataset.queries_iter():
  qry = url + query.text
  x = requests.get(qry)

  if x.status_code == 201:
    obj = json.loads(x.text)
    documents.append({
        "query_id":query.query_id,
        "result":obj
    })

  print("Query ID:", query.query_id, "Status code:", x.status_code)

In [ ]:
for doc in documents[0:1]:
  print(doc) 

In [ ]:
import pandas as pd
pd.DataFrame(dataset.queries_iter())

In [ ]:
dataset.queries_cls()._fields

# RELEVANCIA

In [ ]:
def temRegistro(query_id, document_id, relevance):
  for doc in documents:
    if str(query_id) != doc['query_id']:
      continue
    docx = doc['result']['queryResults']
    i = 0
    for dc in docx:
      i += 1
      if str(document_id) != dc['document']['id']:
        continue
      if str(document_id) == dc['document']['id']:
        return {
            "query_id":query_id,
            "document_id":document_id,
            "relevance":relevance,
            "seq":i
        }


In [ ]:
temRegistro(1,2,2)

In [ ]:
docstore = dataset.docs_store()
docstore.get('486').text

In [ ]:
df = pd.DataFrame(dataset.qrels_iter())
df = df.query("query_id == '1'")

for it in df.values.tolist():
  print(temRegistro(it[0],it[1],it[2]))